# Text classification

🤗 Transformer의 pretraiend model을 활용하여 downstream task를 진행하는 첫번째 시간입니다.

오늘은 Text Classification 중에서 setiment analysis(감정 분석)를 실습해볼 것입니다.
* text classification은 task 중에서 datasets에 등록된 데이터셋은 [nsmc](https://huggingface.co/datasets/nsmc), [3i4k](https://huggingface.co/datasets/kor_3i4k) 등이 존재합니다.
* 3i4k는 의도 분석을 위한 데이터셋으로 오늘 진행하는 감정 분석 task의 코드를 응용하여 구현할 수 있으니 같이 진행해보셔도 좋을 것 같습니다.

활용할 데이터셋은 nsmc입니다.

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.8 MB/s 
     |████████████████████████████████| 120 kB 62.9 MB/s 
     |████████████████████████████████| 6.6 MB 56.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.8 MB/s 
     |████████████████████████████████| 212 kB 58.7 MB/s 
     |████████████████████████████████| 115 kB 65.1 MB/s 
     |████████████████████████████████| 127 kB 69.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from datasets import load_dataset

dataset = load_dataset('nsmc')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## 간단한 데이터 EDA

학습 빠르게 실습해보기위해 데이터를 random sampling할 것입니다.

random sampling하기 전에 간단하게 데이터 EDA를 진행해보겠습니다.

먼저 데이터의 갯수부터 살펴보겠습니다.

In [12]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


train data는 150000개, test data는 50000개인 것을 확인할 수 있습니다.

존재하는 label의 갯수를 확인해보겠습니다. pandas의 DataFrame을 이용합니다.

In [3]:
import pandas as pd

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

In [17]:
train_df.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [18]:
test_df.head(5)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [19]:
print(train_df['label'].unique())
print(test_df['label'].unique())

[0 1]
[1 0]


label은 총 2개로 0과 1인 존재합니다. 0은 negative, 1은 positive를 가리킵니다.

각 label의 갯수를 확인해보겠습니다.

In [20]:
print(train_df['label'].value_counts())

0    75173
1    74827
Name: label, dtype: int64


In [23]:
print(test_df['label'].value_counts())

1    25173
0    24827
Name: label, dtype: int64


label 갯수가 비슷한 것을 확인할 수가 있습니다. 이 경우에는 어느정도 균형잡힌 label 구성이라고 말할 수 있습니다.

다음으로는 random sampling을 진행할 것입니다. label의 균형을 맞춰 학습 데이터 10000개, 테스트 데이터 5000개를 구성하겠습니다.

In [4]:
train_zero_idx = [ i for i, label in enumerate(train_df['label']) if label == 0]
train_one_idx = [ i for i, label in enumerate(train_df['label']) if label == 1]

test_zero_idx = [ i for i, label in enumerate(test_df['label']) if label == 0]
test_one_idx = [ i for i, label in enumerate(test_df['label']) if label == 1]

In [5]:
import random

random.seed(42)

train_sample_idx = random.sample(train_zero_idx, 5000) + random.sample(train_one_idx, 5000)
test_sample_idx = random.sample(test_zero_idx, 2500) + random.sample(test_one_idx, 2500)

In [6]:
train_set = dataset['train'].select(train_sample_idx)
test_set = dataset['test'].select(test_sample_idx)

In [7]:
import numpy as np

print(train_set,'\n')
print("label 0 갯수:",np.sum(np.array(train_set['label']) == 0))
print("label 1 갯수:",np.sum(np.array(train_set['label']) == 1))

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 10000
}) 

label 0 갯수: 5000
label 1 갯수: 5000


In [8]:
print(test_set,'\n')
print("label 0 갯수:",np.sum(np.array(test_set['label']) == 0))
print("label 1 갯수:",np.sum(np.array(test_set['label']) == 1))

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 5000
}) 

label 0 갯수: 2500
label 1 갯수: 2500


random sampling이 잘 된것을 확인할 수 있습니다.

## 데이터 전처리 및 PyTorch Dataset 만들기

먼저 model과 tokenizer를 load합니다. 사용하는 model은 [bert-base-multilingual-cased](https://huggingface.co/bert-base-multilingual-cased)입니다.

In [9]:
MODEL_NAME = 'bert-base-multilingual-cased'

In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

tokenizer가 정상적으로 동작하는 지 체크하겠습니다.

In [11]:
print(train_set['document'][0])
print(tokenizer.tokenize(train_set['document'][0]))

정말실망스런영화,말장난뿐인영화
['정', '##말', '##실', '##망', '##스', '##런', '##영', '##화', ',', '말', '##장', '##난', '##뿐', '##인', '##영', '##화']


해당 text의 경우 띄어쓰기가 올바르게 되어있지않은 것을 확인할 수 있습니다. 띄어쓰기의 경우 tokenization을 하는데 중요한 영향을 끼치기 때문에 띄어쓰기를 할 필요가 있겠습니다. 

하지만 여기서는 데이터를 별도로 수정하지 않고 그대로 사용하겠습니다. 띄어쓰기와 같은 한국어 전처리 라이브러리를 나중에 소개하는 시간을 별도로 가지도록 하겠습니다.

PyTorch Dataset을 만들도록 하겠습니다.

먼저 text를 encoding합니다.

In [12]:
train_encoding = tokenizer(
    train_set['document'],
    padding=True,
    truncation=True,
    return_tensors='pt'
)

test_encoding = tokenizer(
    test_set['document'],
    padding=True,
    truncation=True,
    return_tensors='pt'
)

encoding된 text와 label을 입력받아서 PyTorch Dataset을 만듭니다.

AutoClass를 이용해 불러왔지만 사용한 model은 BertForSequenceClassification이다. 해당 model을 [forward](https://huggingface.co/docs/transformers/v4.21.2/en/model_doc/bert#transformers.BertForSequenceClassification.forward)해 학습하기 위해서는 다음과 같은 요소들이 필요합니다.
1. input_ids
2. attention_mask
3. topken_types_ids
4. labels

1~3번은 tokenizer를 이용해 encoding한 결과입니다. 4번은 random sampled된 데이터의 label을 이용하면 됩니다.

In [13]:
from torch.utils.data import Dataset

class NSMCDataset(Dataset):
    def __init__(self, encoding, label):
        self.encoding = encoding
        self.label = label

    def __getitem__(self,idx):
        data = {k:v[idx] for k, v in self.encoding.items()}
        data['labels'] = self.label[idx]

        return data

    def __len__(self):
        return len(self.label)

In [14]:
train_dataset = NSMCDataset(train_encoding, train_set['label'])
test_dataset = NSMCDataset(test_encoding, test_set['label'])

dataset이 올바르게 만들어졌는지 확인합니다.

In [15]:
print(train_dataset[0])

{'input_ids': tensor([   101,   9670,  89523,  31503,  89292,  12605,  56710,  30858,  18227,
           117,   9251,  13890,  33305, 119020,  12030,  30858,  18227,    102,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,   

In [16]:
print(test_dataset[0])

{'input_ids': tensor([   101,   9356,  75884,   8996,  31605,   8924, 118729,   9715, 119230,
         49742,  42428,    106,    102,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,   

지금까지 Dataset을 이용해 만들었지만 해당 과정을 [🤗 Transformers HOW-TO-GUIDES](https://huggingface.co/docs/transformers/tasks/sequence_classification#preprocess)에서 제공하는 방식을 사용해도됩니다.

## Train, Evaluate

🤗 Transformers에서 제공하는 TrainingArguments, Trainer를 이용해 학습과 평가를 진행할 것입니다.

TrainingArguments에 hyperparameter들을 먼저 정의하고 Trainer을 이용해 본격적인 학습을 시작할 것입니다.

먼저 TrainingArguments를 정의합니다.

In [24]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = './outputs',
    logging_dir = './logs',
    num_train_epochs = 10,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 64,
    logging_steps = 500,
    save_steps=500,
    save_total_limit=2
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


metric으로 accuracy와 f1 score를 사용합니다.

Trainer에서 metric을 별도로 제공해주지 않기 때문에 Trainer의 compute_metrics 인자에 metric을 계산하는 함수를 넘겨주어야합니다.

metric 계산 함수를 먼저 만들겠습니다.
* 해당 함수는 인자를 통해 EvalPrediction 객체를 넘겨 받습니다.
* EvalPrediction은 predictions와 label_ids를 가집니다.
    * predictions: model의 예측값
    * label_ids: label 값
* datasets에서 제공하는 load_metric()을 이용해 accuracy와 f1 score를 계산합니다.

In [18]:
from datasets import load_metric

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    m1 = load_metric('accuracy')
    m2 = load_metric('f1')

    acc = m1.compute(predictions=preds, references=labels)['accuracy']
    f1 = m2.compute(predictions=preds, references=labels)['f1']

    return {'accuracy':acc, 'f1':f1}

Trainer를 정의합니다.

In [19]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [25]:
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics=compute_metrics
)

model을 학습합니다.

In [26]:
trainer.train()

***** Running training *****
  Num examples = 10000
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1570


Step,Training Loss
500,0.347800
1000,0.112300
1500,0.048200


Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-1000
Configuration saved in ./outputs/checkpoint-1000/config.json
Model weights saved in ./outputs/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-1500
Configuration saved in ./outputs/checkpoint-1500/config.json
Model weights saved in ./outputs/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [outputs/checkpoint-500] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1570, training_loss=0.1630935198182513, metrics={'train_runtime': 2021.0982, 'train_samples_per_second': 49.478, 'train_steps_per_second': 0.777, 'total_flos': 6269443116000000.0, 'train_loss': 0.1630935198182513, 'epoch': 10.0})

In [27]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64


{'eval_loss': 1.0615836381912231,
 'eval_accuracy': 0.8128,
 'eval_f1': 0.817258883248731,
 'eval_runtime': 42.2755,
 'eval_samples_per_second': 118.272,
 'eval_steps_per_second': 1.869,
 'epoch': 10.0}